In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import time
tfd = tfp.distributions
import tensorflow.python.ops.numpy_ops.np_config as np_config
np_config.enable_numpy_behavior()

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

#def logsumexp(x):
#  with tf.device('/gpu:0'):
#    c = tf.math.reduce_max(x)
#    return tf.math.exp(x - c + tf.reduce_logsumexp(x-c))

def proposal(currentState, sigma=1., nProps=100, gpu=True):
  dist       = tfd.Normal(loc=0.0,scale=sigma)
  dimensions = tf.size(currentState)
  center     = currentState + dist.sample([dimensions])
  props      = tf.repeat(center, repeats=[nProps], axis=0)
  if gpu:
    with tf.device('/gpu:0'):
      props     += dist.sample([nProps, dimensions])
  else:
    with tf.device('/cpu:0'):
      props     += dist.sample([nProps, dimensions])
  return props

def qpMCMC(propGpu,
               targetGpu,
               comps,
               nIts=1000,
               nProps=10000,
               dims=2,
               precision=1,
               targetAccept=0.65):
  states = tf.Variable(tf.zeros([nIts, dims]))
  accept = tf.Variable(tf.zeros([nIts])) 
  ranks  = tf.Variable(tf.zeros([nIts]))
  
  Acceptances = 0 # total acceptances within adaptation run (<= SampBound)
  SampBound = 5   # current total samples before adapting radius
  SampCount = 0   # number of samples collected (adapt when = SampBound)

  centers = tf.constant(name='centers', # get locations of mixture of gaussians
                        value=tf.cast(tf.range(0,comps*10,10),
                                      dtype=tf.float32),shape=[1,comps])
  locs    = tf.ones([comps,dims]) * centers.transpose()
  
  for i in range(1,nIts):
    props           = proposal(currentState=[tf.gather(states,i-1)],
                               nProps=nProps,
                               sigma=1/tf.math.sqrt(precision),
                               gpu=propGpu)
    currentAndProps = tf.concat(axis=0,values=[[tf.gather(states,i-1)],props])
    selectionProbs  = -target(currentAndProps, locations=locs,
                             gpu=targetGpu, components=comps)
    dist            = tfd.Gumbel(loc=0., scale=1.)
    with tf.device('/gpu:0'):
      selectionProbs -= dist.sample([nProps+1])
    ranks[i-1].assign(tf.cast(tf.reduce_sum(1*tf.math.less(selectionProbs,selectionProbs[0])),dtype=tf.float32))
    propIndex       = tf.argmin(selectionProbs)
    states[i,:].assign(tf.squeeze(tf.gather(currentAndProps,propIndex)))
    SampCount = SampCount + 1
    
    if propIndex != 0:
      accept[i].assign(1)
      Acceptances = Acceptances + 1    
    if i % 1000 == 0:
       tf.print("Iteration ", i, " complete.\n")
    if SampCount == SampBound: # adjust lambda at increasing intervals
      AcceptRatio = Acceptances / SampBound
      if AcceptRatio == 0:
        AdaptRatio  = 2
      else:
        AdaptRatio  = targetAccept / AcceptRatio
      if AdaptRatio > 2:
        AdaptRatio = 2
      if AdaptRatio < 0.5:
        AdaptRatio = 0.5
      precision *= AdaptRatio
      
      SampCount = 0
      SampBound = tf.math.ceil(tf.math.pow(tf.cast(SampBound,dtype=tf.float32),
                                           1.01))
      Acceptances = 0 
    if i > 100 and i % 1000 == 0:
      ess0 = tfp.mcmc.effective_sample_size(states[99:(i-1),:])
      print(ess0)
      if tf.math.minimum(x=ess0[0],y=ess0[1]) > 100. or i>2000000:
        print(tf.math.minimum(x=ess0[0],y=ess0[1]))
        return [i,ranks[1:(i-1)]]


def target(x, components, locations, gpu=True):

  bimix_gauss = tfd.MixtureSameFamily(
  mixture_distribution=tfd.Categorical(probs=tf.ones([components])),
  components_distribution=tfd.MultivariateNormalDiag(loc=locations))
  if gpu:
    with tf.device('/gpu:0'):
      return bimix_gauss.log_prob(x)
  else:
    with tf.device('/cpu:0'):
      return bimix_gauss.log_prob(x)

# def expoSearch(marks):
#   success  = False
#   m        = 1
#   grovIter = 0
#   lmbd     = 6/5
#   M        = tf.math.reduce_sum(1*(marks==1))
#   N        = tf.size(marks)
#   while (not success) and grovIter<(9*tf.math.sqrt(tf.cast(N,dtype=tf.float32))/4):
#     dist      = tfd.Categorical(probs=tf.ones([m])/m)
#     j         = dist.sample([1])
#     sqrtProbs = tf.ones([N])/tf.math.sqrt(tf.cast(N,dtype=tf.float32))
#     i         = 1
#     while i <= j:
#       sqrtProbs  = -marks*sqrtProbs + (1-marks)*sqrtProbs
#       sqrtProbs  = -sqrtProbs + 2*tf.reduce_mean(sqrtProbs)
#       i         += 1
#     dist = tfd.Categorical(probs=tf.math.pow(sqrtProbs,2))
#     draw = dist.sample([1]) + 1
#     grovIter += j
#     if draw <= M:
#       success = True
#     else:
#       m = tf.math.minimum(lmbd*m,tf.math.sqrt(tf.cast(N,dtype=tf.float32)))

#   return [grovIter,draw,success]

# def quantumMin(field, y):
#   done        = False
#   oracleCalls = 0
#   while not done:
#     marks        = 1*tf.math.less(field,field[y])
#     res          = expoSearch(marks)
#     oracleCalls += res[0]
#     if res[2]:
#       y = res[1]
#     else:
#       done = True
#   return([y,oracleCalls])



Found GPU at: /device:GPU:0


Expo search test.

In [2]:
rng = tf.random.Generator.from_seed(seed=1, alg="philox")
nIts = 10000000
comps = 1000
d = 2

for nProps in [10000,5000,1000]:
  for k in tf.range(5):
      gpuOut = qpMCMC(propGpu=True, targetGpu=True, comps=comps,
              nIts=nIts,nProps=nProps,dims=d, targetAccept=0.50,
              precision=2.0*tf.math.sqrt(tf.cast(d,dtype=tf.float32)))

      with open('/content/drive/MyDrive/bbMCMC_output/qpStudy.txt', 'a') as f:
        print(nProps, gpuOut[0], file=f)
        f.close()

      if nProps == 10000:
        np.savetxt('/content/drive/MyDrive/bbMCMC_output/qpMCMC10KRanks'+tf.strings.as_string(k).numpy().decode()+'.txt',
                 gpuOut[1].numpy(),delimiter="\n\n")
      
      if nProps == 5000:
        np.savetxt('/content/drive/MyDrive/bbMCMC_output/qpMCMC5KRanks'+tf.strings.as_string(k).numpy().decode()+'.txt',
                 gpuOut[1].numpy(),delimiter="\n\n")
      
      if nProps == 1000:
        np.savetxt('/content/drive/MyDrive/bbMCMC_output/qpMCMC1KRanks'+tf.strings.as_string(k).numpy().decode()+'.txt',
                 gpuOut[1].numpy(),delimiter="\n\n")

Iteration  1000  complete.

tf.Tensor([5.1562147 5.156122 ], shape=(2,), dtype=float32)
Iteration  2000  complete.

tf.Tensor([14.31193   14.3115635], shape=(2,), dtype=float32)
Iteration  3000  complete.

tf.Tensor([34.019897 34.019215], shape=(2,), dtype=float32)
Iteration  4000  complete.

tf.Tensor([53.11577  53.114727], shape=(2,), dtype=float32)
Iteration  5000  complete.

tf.Tensor([63.50887  63.503826], shape=(2,), dtype=float32)
Iteration  6000  complete.

tf.Tensor([90.80033 90.79797], shape=(2,), dtype=float32)
Iteration  7000  complete.

tf.Tensor([105.50243 105.49698], shape=(2,), dtype=float32)
tf.Tensor(105.49698, shape=(), dtype=float32)
Iteration  1000  complete.

tf.Tensor([9.551675 9.551849], shape=(2,), dtype=float32)
Iteration  2000  complete.

tf.Tensor([25.432026 25.431696], shape=(2,), dtype=float32)
Iteration  3000  complete.

tf.Tensor([42.6837  42.68397], shape=(2,), dtype=float32)
Iteration  4000  complete.

tf.Tensor([64.69407 64.69492], shape=(2,), dtype=f